In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage

##Filters
A Filter is a function from an asset and a moment in time to a boolean:
```
F(asset, timestamp) -> boolean
```
In Pipeline, [Filters](https://www.quantopian.com/help#quantopian_pipeline_filters_Filter) are used for narrowing down the set of securities included in a computation or in the final output of a pipeline. There are two common ways to create a `Filter`: comparison operators and `Factor`/`Classifier` methods.

###Comparison Operators
Comparison operators on `Factors` and `Classifiers` produce Filters. For now, let's stick to examples using `Factors`. The following example produces a filter that returns `True` whenever the latest close price is above $20.

In [ ]:
last_close_price = USEquityPricing.close.latest
close_price_filter = last_close_price > 20

This example produces a filter that returns True whenever the 10-day mean is below the 30-day mean.

In [ ]:
mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)
mean_crossover_filter = mean_close_10 < mean_close_30

Each security will get its own `True` or `False` value each day.

###Factor/Classifier Methods
Various methods of the `Factor` and `Classifier` classes return `Filters`. For now, let's stick to `Factor` methods. The `Factor.top(n)` method produces a `Filter` that returns `True` for the top `n` securities of a given `Factor`. The following cell produces a filter that returns `True` for exactly 200 securities every day, indicating that those securities were in the top 200 by last close price across all known securities.

In [ ]:
last_close_price = USEquityPricing.close.latest
top_close_price_filter = last_close_price.top(200)

##Dollar Volume Filter
As a starting example, let's create a filter that returns `True` if a security's 30-day average dollar volume is above $10,000,000. To do this, you first need to create an `AverageDollarVolume` factor to compute the 30-day average dollar volume. Let's include the built-in `AverageDollarVolume` factor in the imports:

In [ ]:
from quantopian.pipeline.factors import AverageDollarVolume

Let's instantiate the average dollar volume factor.

In [ ]:
dollar_volume = AverageDollarVolume(window_length=30)

By default, `AverageDollarVolume` uses `USEquityPricing.close` and `USEquityPricing.volume` as its `inputs`, so you don't have to specify them.

Now that there is a dollar volume factor, you can create a filter with a boolean expression. The following line creates a filter returning `True` for securities with a `dollar_volume` greater than 10,000,000:

In [ ]:
high_dollar_volume = (dollar_volume > 10000000)

To see what the filter looks like, let's add it as a column to the pipeline, defined in the previous lesson.

In [ ]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30
    
    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = (dollar_volume > 10000000)

    return Pipeline(
        columns={
            'percent_difference': percent_difference,
            'high_dollar_volume': high_dollar_volume
        }
    )

If you make and run the pipeline, you now have a column `high_dollar_volume` with a boolean value corresponding to the result of the expression for each security.

In [ ]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

high_dollar_volume  \
2015-05-05 00:00:00+00:00 Equity(2 [HWM])                        True   
                          Equity(21 [AAME])                     False   
                          Equity(24 [AAPL])                      True   
                          Equity(25 [HWM_PR])                   False   
                          Equity(31 [ABAX])                     False   
                          Equity(39 [DDC])                      False   
                          Equity(41 [ARCB])                     False   
                          Equity(52 [ABM])                      False   
                          Equity(53 [ABMD])                      True   
                          Equity(62 [ABT])                       True   
                          Equity(64 [GOLD])                      True   
                          Equity(66 [AB])                       False   
                          Equity(67 [ADSK])                      True   
                          Equity(69 [ACAT])                     False   
                          Equity(70 [VBF])                      False   
                          Equity(76 [TAP])                       True   
                          Equity(84 [ACET])                     False   
                          Equity(86 [ACG])                      False   
                          Equity(88 [ACI])                      False   
                          Equity(100 [IEP])                     False   
                          Equity(106 [ACU])                     False   
                          Equity(110 [RAMP])                    False   
                          Equity(112 [ACY])                     False   
                          Equity(114 [ADBE])                     True   
                          Equity(117 [AEY])                     False   
                          Equity(122 [ADI])                      True   
                          Equity(128 [ADM])                      True   
                          Equity(134 [SXCL])                    False   
                          Equity(149 [ADX])                     False   
                          Equity(153 [AE])                      False   
...                                                               ...   
                          Equity(48945 [HLTH])                  False   
                          Equity(48954 [INF_RT])                False   
                          Equity(48955 [ALIN_PRB])              False   
                          Equity(48960 [SMCP])                  False   
                          Equity(48961 [NYMT_O])                False   
                          Equity(48962 [UNIT])                   True   
                          Equity(48963 [PAK])                   False   
                          Equity(48969 [NSA])                   False   
                          Equity(48971 [BSM])                    True   
                          Equity(48972 [EVA])                   False   
                          Equity(48981 [APIC])                  False   
                          Equity(48989 [UK])                    False   
                          Equity(48990 [ACWF])                  False   
                          Equity(48991 [ISCF])                  False   
                          Equity(48992 [INTF])                  False   
                          Equity(48993 [JETS])                  False   
                          Equity(48994 [ACTX])                  False   
                          Equity(48995 [LRGF])                  False   
                          Equity(48996 [SMLF])                  False   
                          Equity(48997 [VKTX])                  False   
                          Equity(48998 [OPGN])                  False   
                          Equity(48999 [AAPC])                  False   
                          Equity(49000 [BPMC])                  False   
                          Equity(490

##Applying a Screen
By default, a pipeline produces computed values each day for every asset in the Quantopian database. Very often however, you only care about a subset of securities that meet specific criteria (for example, we might only care about securities that have enough daily trading volume to fill orders quickly). You can tell the Pipeline to ignore securities for which a filter produces `False` by passing that filter to the Pipeline via the `screen` keyword.

To screen the pipeline output for securities with a 30-day average dollar volume greater than $10,000,000, you can simply pass the `high_dollar_volume` filter as the `screen` argument. This is what the `make_pipeline` function now looks like:

In [ ]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume > 10000000

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=high_dollar_volume
    )

When you run this, the pipeline output only includes securities that pass the `high_dollar_volume` filter on a given day. For example, running the pipeline on May 5th, 2015 results in an output for ~2,100 securities

In [ ]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result

Number of securities that passed the filter: 2106


percent_difference
2015-05-05 00:00:00+00:00 Equity(2 [HWM])                      0.017975
                          Equity(24 [AAPL])                    0.016905
                          Equity(53 [ABMD])                   -0.024682
                          Equity(62 [ABT])                     0.014385
                          Equity(64 [GOLD])                    0.046963
                          Equity(67 [ADSK])                   -0.003921
                          Equity(76 [TAP])                    -0.008759
                          Equity(114 [ADBE])                   0.009499
                          Equity(122 [ADI])                    0.009271
                          Equity(128 [ADM])                    0.015760
                          Equity(154 [AEM])                    0.026035
                          Equity(161 [AEP])                    0.010405
                          Equity(166 [AES])                    0.022158
                          Equity(168 [AET])                    0.005853
                          Equity(185 [AFL])                   -0.002239
                          Equity(197 [AGCO])                   0.032124
                          Equity(216 [HES])                    0.036528
                          Equity(239 [AIG])                    0.012322
                          Equity(253 [AIR])                   -0.012412
                          Equity(266 [AJG])                    0.012267
                          Equity(270 [AKRX])                  -0.024963
                          Equity(273 [ALU])                   -0.021750
                          Equity(300 [ALK])                    0.015147
                          Equity(301 [ALKS])                  -0.033228
                          Equity(328 [ALTR])                   0.012284
                          Equity(337 [AMAT])                  -0.050162
                          Equity(351 [AMD])                   -0.101477
                          Equity(353 [AME])                   -0.003008
                          Equity(357 [TWX])                    0.000365
                          Equity(368 [AMGN])                   0.008860
...                                                                 ...
                          Equity(48091 [VA])                  -0.005665
                          Equity(48103 [STOR])                -0.045272
                          Equity(48104 [PGRE])                -0.008569
                          Equity(48126 [HABT])                 0.063080
                          Equity(48129 [UBS])                  0.025888
                          Equity(48169 [KLXI])                 0.021062
                          Equity(48215 [QSR])                  0.037460
                          Equity(48220 [LC])                  -0.035048
                          Equity(48317 [JUNO])                -0.103370
                          Equity(48384 [QRVO])                -0.050578
                          Equity(48465 [SWNC])                 0.061669
                          Equity(48486 [BOX])                 -0.003837
                          Equity(48531 [VSTO])                 0.017196
                          Equity(48543 [SHAK])                 0.175877
                          Equity(48544 [HIFR])                 0.027339
                          Equity(48547 [ONCE])                -0.112191
                          Equity(48575 [XHR])                 -0.008521
                          Equity(48629 [INOV])                -0.068366
                          Equity(48662 [JPM_PRFCL])            0.002978
                          Equity(48672 [TOTL])                 0.000991
                          Equity(48730 [AGN_PRA])             -0.008843
                          Equity(48821 [CJES])                 0.099492
                          Equity(48823 [SEDG])                 0.056643
                          Equity(48863 [GDDY])                -0.003563
                     

##Inverting a Filter
The `~` operator is used to invert a filter, swapping all `True` values with `Falses` and vice-versa. For example, you can write the following to filter for low dollar volume securities:

In [ ]:
low_dollar_volume = ~high_dollar_volume

This will return `True` for all securities with an average dollar volume below or equal to $10,000,000 over the last 30 days.